In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
from textSummarizer.logging import logger
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
%pwd

'd:\\Data Science\\Text-summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data Science\\Text-summarizer'

In [5]:
@dataclass(frozen=True)
class DataTransfromationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from textSummarizer.utils.common import read_yaml_config,create_directorys

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_config(config_filepath)
        self.params = read_yaml_config(params_filepath)

        create_directorys([self.config.artifacts_root]) # config hold a ConfigBox type so we can access its attribites by just config.attribute
    
    def get_data_transformation_config(self) -> DataTransfromationConfig:
        config = self.config.data_transformation

        create_directorys([config.root_dir])

        data_transformation_config = DataTransfromationConfig(
            root_dir = config.root_dir,
            data_path= config.data_path,
            tokenizer_name= config.tokenizer_name,
        )
        return data_transformation_config


In [8]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

d:\Data Science\Text-summarizer\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-08-18 13:17:12,611: INFO: config: PyTorch version 2.3.1 available.]


In [9]:
#component
class DataTransfromation:
    def __init__(self, config: DataTransfromationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_to_features(self, example_batche):
        input_encodings = self.tokenizer(example_batche['dialogue'], truncation=True, max_length=1024)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batche['summary'], truncation=True,max_length=128)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids'],
        }
        
    def convert(self):
        dataset_samsum =  load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))

In [10]:
# pipeline
try:
    config = ConfigurationManager()
    data_transformatiom_config = config.get_data_transformation_config()
    data_transformatiom = DataTransfromation(config=data_transformatiom_config)
    data_transformatiom.convert()
except Exception as e:
    raise e

[2024-08-18 13:17:16,424: INFO: common: Successfully loaded configuration from config\config.yaml]
[2024-08-18 13:17:16,427: INFO: common: Successfully loaded configuration from params.yaml]
[2024-08-18 13:17:16,428: INFO: common: Created directory: artifacts]
[2024-08-18 13:17:16,430: INFO: common: Created directory: artifacts/data_transformation]


d:\Data Science\Text-summarizer\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\Data Science\Text-summarizer\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 44845.38 examples/s]
